In [1]:
import mappy as mp
import pandas as pd
import re,tempfile,shutil,os,argparse,subprocess
from subprocess import call
from multiprocessing import Pool
import itertools
import gzip

# Get pattern automatically

In [103]:
def find_insert_position(read_pair_1, read_pair_2, align_pattern):
    insert_position = 0

    def align(seq_1, seq_2):
        align_base = ['1' if base1 == base2 else '0' for base1,
                      base2 in zip(seq_1, seq_2)]
        return ''.join(align_base)

    for i, reada in enumerate(read_pair_1):
        for j, readb in enumerate(read_pair_2):
            align_result = align(reada, readb)
        if '1'*len(readb) == align(reada, readb):
            sgrna_align = align(read_pair_1[j], read_pair_2[i])
            match = re.findall(align_pattern, sgrna_align)
            if match:
                position = len(match[0][0])
                insert_position = [position,
                                   read_pair_1[j][position-4:position],
                                   read_pair_1[j][position+20:position+24]]
            break
    return insert_position

In [104]:
def get_common_pattern(fq1, fq2):
    ith = 0
    seq = {}
    for read1, read2 in zip(mp.fastx_read(fq1, read_comment=True),
                            mp.fastx_read(fq2, read_comment=True)):
        seq[read1[0]] = [read1[1], read2[1]]
        ith += 1
        if ith == 1000:
            break

    detect_df = pd.DataFrame(columns=['position', 'left', 'right'])
    align_pattern = re.compile(r'(^1+)0([0-1]{19})(1+$)')
    ith = 0
    for reada_id, readb_id in itertools.combinations(seq.keys(), 2):
        insert_position = find_insert_position(
            seq[reada_id], seq[readb_id], align_pattern)
        if insert_position:
            detect_df.loc[ith] = insert_position
            ith += 1
            if ith == 100:
                break
    return detect_df

In [110]:
fq1,fq2 = '/f/jianhua/LibraryQC/input/data/vf_1.fq.gz','/f/jianhua/LibraryQC/input/data/vf_2.fq.gz'
common_pattern = get_common_pattern(fq1,fq2)
common_pattern.mode()

,position,left,right
0,50,AAAC,CGGT
